# Museums in the Pandemic - Extract indicators

**Authors**: Andrea Ballatore (KCL)

**Abstract**: Extract indicators from museum text.

## Setup
This is to check that your environment is set up correctly (it should print 'env ok', ignore warnings).

In [1]:
# Test geospatial libraries
# check environment
import os
print("Conda env:", os.environ['CONDA_DEFAULT_ENV'])
if os.environ['CONDA_DEFAULT_ENV'] != 'mip_v1':
    raise Exception("Set the environment 'mip_v1' on Anaconda. Current environment: " + os.environ['CONDA_DEFAULT_ENV'])

# spatial libraries 
import pandas as pd
import pickle
import spacy
from termcolor import colored
import sys
import numpy as np
from numpy import arange
#import tensorflow as tf
from bs4 import BeautifulSoup
from bs4.element import Comment
#import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# import from `mip` project
print(os.getcwd())
fpath = os.path.abspath('../')
if not fpath in sys.path:
    sys.path.insert(0, fpath)

out_folder = '../../'

from museums import *
from utils import _is_number

print('env ok')

Conda env: mip_v1
/Users/andreaballatore/Dropbox/DRBX_Docs/Work/Projects/github_projects/museums-in-the-pandemic/mip/notebooks
env ok


## Connect to DB

It needs the DCS VPN active to work.

In [3]:
# open connection to DB
from db.db import connect_to_postgresql_db

db_conn = connect_to_postgresql_db()
print("DB connected")

DB connected


## Extract matches for all museums

Using the best deep learning model defined above, find indicators for all museums (from websites and social media).

### Load deep learning validation model

In [4]:
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

def prep_match_data(df):
    num_df = valid_ann_df.select_dtypes(include=[np.number])
    scaler = MinMaxScaler()
    # fit and transform in one step
    cols = num_df.columns
    x_data = pd.DataFrame(scaler.fit_transform(num_df),columns=cols)
    return x_data

def convert_pred_to_bool(vals):
    pred_y = (vals > 0.5).astype("bool")
    # unpack results
    bool_vals = [item for sublist in pred_y for item in sublist]
    return bool_vals

valid_ann_df_fn = 'matches_valid_ann_df_v2.pik'
valid_ann_df = pd.read_pickle(out_folder+'data/annotations/'+valid_ann_df_fn)

valid_match_cnn_model = load_model(out_folder+"data/analysis/matching_validation/matching_validation_deep_learning_model.h5")
valid_match_cnn_model

#x_data = prep_match_data(valid_ann_df)
#pred_valid = convert_pred_to_bool(valid_match_cnn_model.predict(x_data))

#valid_ann_df['predicted_valid'] = pred_valid

#valid_ann_df.to_excel(out_folder+"tmp/check_deeplearning.xlsx",index=False)
#valid_ann_df.sample(5)

2021-10-11 21:13:07.004195: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load all matches from DB for all museums

- Dump all matches from DB, after running `an_text` on ALL museums for a given crawling session.

In [5]:
# load from DB
def get_all_matches_from_db(session_id):
    print("get_all_matches_from_db", session_id)
    sql = """select muse_id, example_id, sentence_id, indicator_code, token_n, lemma_n, 
                    ann_overlap_lemma, ann_overlap_token, example_len, ann_ex_tokens, page_tokens,
                    txt_overlap_lemma, txt_overlap_token, ann_overlap_criticwords
        from analytics.text_indic_ann_matches_{} t where keep_stopwords;""".format(session_id)
    df = pd.read_sql(sql, db_conn)
    print("rows",len(df))

    # write df to file
    matches_fn = out_folder+'tmp/matches_dump_df_{}.pik'.format(session_id)
    df.to_pickle(matches_fn)
    del df
    print('\tsaved',fn)
    return matches_fn

sessions = ['20210304','20210404']
for session_id in sessions:
    get_all_matches_from_db(session_id)

get_all_matches_from_db 20210304
rows 7167891


### Predict all matches

In [ ]:
TODO
#x_data = prep_match_data(valid_ann_df)
#pred_valid = convert_pred_to_bool(valid_match_cnn_model.predict(x_data))


In [ ]:
### extract stable results

End of notebook